In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta


def fetch_binance_data(symbol, interval, start_time, end_time):
    base_url = "https://api.binance.com/api/v3/klines"

    start_ms = int(start_time.timestamp() * 1000)
    end_ms = int(end_time.timestamp() * 1000)

    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": start_ms,
        "endTime": end_ms,
        "limit": 1000
    }

    all_klines = []
    while start_ms < end_ms:
        response = requests.get(base_url, params=params)
        klines = response.json()

        if not klines:
            break

        all_klines.extend(klines)
        start_ms = klines[-1][0] + 1
        params["startTime"] = start_ms

    df = pd.DataFrame(all_klines, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ])

    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[col] = df[col].astype(float)

    return df[['open', 'high', 'low', 'close', 'volume']]


# specific to wira
def resample_ohlcv(df, timeframe):
    resampled = df.resample(timeframe)
    return pd.DataFrame({
        'open': resampled['open'].first(),
        'high': resampled['high'].max(),
        'low': resampled['low'].min(),
        'close': resampled['close'].last(),
        'volume': resampled['volume'].sum()
    })

# specific to wira strategy
rolling_window = 30
fee_rate = 0.0002
initial_balance = 10000
z_score_threshold = 0.4
leverage = 10
margin_call_threshold = 0.3

# set_parameters = {
# rolling_window = [30, 60, 90, 120]
# fee_rate = 0.0002
# initial_balance = 10000
# z_score_threshold = [-0.8, -0.3, 0, 0.3, 0.4, 0.8]
# leverage = [10, 5, 2]
# margin_call_threshold = 0.3}

class Trade:
    def __init__(self, entry_time, symbol_a, symbol_b, position, entry_price_a, entry_price_b, amount_a, amount_b):
        self.entry_time = entry_time
        self.symbol_a = symbol_a
        self.symbol_b = symbol_b
        self.position = position  # 1 for long A/short B, -1 for short A/long B
        self.entry_price_a = entry_price_a
        self.entry_price_b = entry_price_b
        self.amount_a = amount_a
        self.amount_b = amount_b
        self.exit_time = None
        self.exit_price_a = None
        self.exit_price_b = None
        self.pnl_a = None
        self.pnl_b = None
        self.total_pnl = None

    def close_trade(self, exit_time, exit_price_a, exit_price_b):
        self.exit_time = exit_time
        self.exit_price_a = exit_price_a
        self.exit_price_b = exit_price_b
        self.calculate_pnl()

    def calculate_pnl(self):
        if self.position == 1:  # Long A, Short B
            self.pnl_a = (self.exit_price_a - self.entry_price_a) * self.amount_a
            self.pnl_b = (self.entry_price_b - self.exit_price_b) * self.amount_b
        else:  # Short A, Long B
            self.pnl_a = (self.entry_price_a - self.exit_price_a) * self.amount_a
            self.pnl_b = (self.exit_price_b - self.entry_price_b) * self.amount_b
        self.total_pnl = self.pnl_a + self.pnl_b


# Finding signals
def update_strategy(data_a, data_b, symbol_a, symbol_b):
    data = pd.DataFrame({
        symbol_a: data_a['close'],
        symbol_b: data_b['close']
    })

    pct_change = data.pct_change()

    crypto_autocorr = pct_change.rolling(rolling_window).apply(lambda x: x.autocorr())
    crypto_autocorr_signal = -(crypto_autocorr.subtract(crypto_autocorr.mean(axis=1), axis=0))
    crypto_autocorr_signal = crypto_autocorr_signal.divide(crypto_autocorr_signal.abs().sum(axis=1), axis=0)
    crypto_autocorr_signal = crypto_autocorr_signal.rename(columns={symbol_a: "AutoCorr_A", symbol_b: "AutoCorr_B"})

    data['zscoreA'] = (data[symbol_a] - data[symbol_a].rolling(rolling_window).mean()) / data[symbol_a].rolling(
        rolling_window).std()
    data['zscoreB'] = (data[symbol_b] - data[symbol_b].rolling(rolling_window).mean()) / data[symbol_b].rolling(
        rolling_window).std()
    data['zscoreSpread'] = data['zscoreA'] - data['zscoreB']

    data['signal'] = 0
    data["AutoCorr_A"] = crypto_autocorr_signal["AutoCorr_A"]
    data["AutoCorr_B"] = crypto_autocorr_signal["AutoCorr_B"]
    data.loc[(data["AutoCorr_A"] < data["AutoCorr_B"]) & (data['zscoreSpread'] < -z_score_threshold), 'signal'] = 1
    data.loc[(data["AutoCorr_A"] > data["AutoCorr_B"]) & (data['zscoreSpread'] > z_score_threshold), 'signal'] = -1

    data['position'] = data['signal'].replace(0, np.nan).ffill()
    data['position_change'] = data['position'].diff()

    trades = []
    current_trade = None

    for index, row in data.iterrows():
        if row['position_change'] != 0:
            if current_trade is not None:
                current_trade.close_trade(index, row[symbol_a], row[symbol_b])
                trades.append(current_trade)

            if row['position'] != 0:
                trade_value = initial_balance * leverage / 2
                amount_a = trade_value / row[symbol_a]
                amount_b = trade_value / row[symbol_b]
                current_trade = Trade(index, symbol_a, symbol_b, row['position'], row[symbol_a], row[symbol_b],
                                      amount_a, amount_b)

    if current_trade is not None:
        current_trade.close_trade(data.index[-1], data[symbol_a].iloc[-1], data[symbol_b].iloc[-1])
        trades.append(current_trade)

    return data, trades


# Main execution
if __name__ == "__main__":
    # Fetch and process data for the last 2 weeks
    end_time = datetime.now()
    start_time = end_time - timedelta(days=14)

    # Configure your pairs here
    symbol_a = "ALICEUSDT"
    symbol_b = "TLMUSDT"

    data_a = fetch_binance_data(symbol_a, "5m", start_time, end_time)
    data_b = fetch_binance_data(symbol_b, "5m", start_time, end_time)

    data_a_10m = resample_ohlcv(data_a, '10T')
    data_b_10m = resample_ohlcv(data_b, '10T')

    # Update strategy
    result, trades = update_strategy(data_a_10m, data_b_10m, symbol_a, symbol_b)

    # Print trade list with PNL
    print(f"Trade List for the Last 2 Weeks ({symbol_a} vs {symbol_b}):")
    total_pnl = 0
    for i, trade in enumerate(trades, 1):
        print(f"Trade {i}:")
        print(f"Entry Time: {trade.entry_time}")
        print(f"Exit Time: {trade.exit_time}")
        print(
            f"Position: {'Long ' + trade.symbol_a + '/Short ' + trade.symbol_b if trade.position == 1 else 'Short ' + trade.symbol_a + '/Long ' + trade.symbol_b}")
        print(
            f"{trade.symbol_a}: Entry = {trade.entry_price_a:.4f}, Exit = {trade.exit_price_a:.4f}, Amount = {trade.amount_a:.6f}")
        print(
            f"{trade.symbol_b}: Entry = {trade.entry_price_b:.4f}, Exit = {trade.exit_price_b:.4f}, Amount = {trade.amount_b:.6f}")
        print(f"PNL {trade.symbol_a}: ${trade.pnl_a:.2f}")
        print(f"PNL {trade.symbol_b}: ${trade.pnl_b:.2f}")
        print(f"Total PNL: ${trade.total_pnl:.2f}")
        print("-" * 50)
        total_pnl += trade.total_pnl

    # Calculate and print summary statistics
    total_trades = len(trades)
    total_fees = total_trades * initial_balance * leverage * fee_rate * 2  # 2 because each trade involves two assets
    profit_after_fees = total_pnl - total_fees
    return_after_fees = profit_after_fees / initial_balance

    print("\nSummary:")
    print(f"Total number of trades: {total_trades}")
    print(f"First trade time: {trades[0].entry_time if trades else 'No trades'}")
    print(f"Last trade time: {trades[-1].exit_time if trades else 'No trades'}")
    print(f"Total PNL (before fees): ${total_pnl:.2f}")
    print(f"Total fees: ${total_fees:.2f}")
    print(f"Total profit (after fees): ${profit_after_fees:.2f}")
    print(f"Total return (after fees): {return_after_fees:.2%}")

    # Calculate max drawdown
    cumulative_pnl = np.cumsum([trade.total_pnl for trade in trades])
    max_drawdown = (cumulative_pnl - np.maximum.accumulate(cumulative_pnl)).min() / initial_balance
    print(f"Max drawdown: {max_drawdown:.2%}")

SSLError: HTTPSConnectionPool(host='api.binance.com', port=443): Max retries exceeded with url: /api/v3/klines?symbol=ALICEUSDT&interval=5m&startTime=1728441735611&endTime=1729651335611&limit=1000 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))